![](https://ai-studio-static-online.cdn.bcebos.com/ef9f89df07cb4c22a11086a488af62ddefb6399fa97d467da9e0471035265f32)


	在本次赛题中，虽然赛题是一个二分类任务（用户购买、未购买），但从赛题数据看，属于比较典型的时间序列数据，也可以参照以往的线性回归任务的做法处理。 反映了一个客观事实——在真实场景应用机器学习/深度学习技术时，通常是没有已经整理好的训练集、验证集、测试集，需要自己设计。
	XGBoost（eXtreme Gradient Boosting）是一种流行的机器学习算法，用于分类和回归问题。它是一种集成学习方法，即通过训练多个弱学习器（例如决策树）并将它们组合在一起，形成一个更强大的模型。XGBoost的优势在于它能够快速地学习特征重要性，并使用决策树算法构建高质量的模型。它还使用了一些技术来提高训练效率，包括基于决策树的正则化、直方图算法和并行训练。XGBoost通常用于竞赛和实际应用中，因为它能够在大型数据集上快速构建高质量的模型，能够较好地处理缺失数据并在高维度空间中执行优秀的预测。


数据如下图所示

![](https://ai-studio-static-online.cdn.bcebos.com/d997a498b4da47b49178d7f0c0f22581e1941200d9b44863af66131611a29d6b)


	部分数据，如会员id、用户id等数据，应该剔除，所以要对原始数据进行预处理，进行特征的提取与剔除。由于数据集中的数据属于时间序列数据，可以引入时间滑窗模型。时间滑窗在业务应用上被称为RFM模型，RFM（Recency, Frequency, Monetary Value）模型是一种客户分析方法，用于衡量客户的价值和潜在收益。它通过对客户的最近一次交易时间（Recency）、交易频率（Frequency）以及交易金额（Monetary Value）进行评估，并将客户分为不同的类别，以便更好地了解客户的购买行为，并采取相应的营销策略。在RFM模型中，分数越高的客户被认为是更有价值的客户。因此，对于分数较低的客户，企业可以采取一些措施来提高他们的分数，并将他们转变为更有价值的客户。例如，企业可以通过给予他们优惠或促销活动来吸引他们的注意，或者通过提供优质的产品和服务来提高他们的满意度，从而增加他们的交易频率和金额。
	本实验采用RFM模型，考虑到按周、按月的订单规律性变化，时间滑窗往往是14和30的倍数，一般选取的特征是滑窗中的基本统计值，最大、最小、均值、中位数、求和等等。


# 引入工具包

In [1]:
import numpy as np  # linear algebra
import pandas as pd  #
from datetime import datetime, date, timedelta
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,Ridge,Lasso,ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor,RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.svm import SVR, LinearSVC
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
import time
from itertools import product
import datetime as dt
import calendar
import gc

RANDOM_SEED = 42
# 引入中文字体
from matplotlib.font_manager import FontProperties
myfont = FontProperties(fname="/home/aistudio/NotoSansCJKsc-Light.otf", size=12)

# 加载数据集

In [2]:
PATH = './data/data19383/'
train = pd.read_csv(PATH + 'train.csv')
# train = pd.read_csv('./data/data19383/train.csv', usecols=[2, 3, 4, 6, 7, 18])
# set index to ID to avoid droping it later
# 把测试集的id列作为索引，防止误删
test  = pd.read_csv(PATH + 'submission.csv').set_index('customer_id')

In [3]:
# 对于特别大的文件，我们需要做一些内存检查
mem_train = train.memory_usage(index=True).sum()
mem_test=test.memory_usage(index=True).sum()
print(u"训练集使用内容 "+ str(mem_train/ 1024**2)+" MB")
print(u"测试集使用内存 "+ str(mem_test/ 1024**2)+" MB")

训练集使用内容 510.40096282958984 MB
测试集使用内存 24.200225830078125 MB


### 内存优化脚本
- 参考[缓解pandas中DataFrame占用内存过大的问题](https://blog.csdn.net/wj1066/article/details/81124959)
- 效果非常显著，有效避免内存溢出

In [4]:
# @from: https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65/code
# @liscense: Apache 2.0
# @author: weijian
def reduce_mem_usage(props):
    # 计算当前内存
    start_mem_usg = props.memory_usage().sum() / 1024 ** 2
    print("Memory usage of the dataframe is :", start_mem_usg, "MB")
    
    # 哪些列包含空值，空值用-999填充。why：因为np.nan当做float处理
    NAlist = []
    for col in props.columns:
        # 这里只过滤了objectd格式，如果你的代码中还包含其他类型，请一并过滤
        if (props[col].dtypes != object):
            
            # print("**************************")
            # print("columns: ", col)
            # print("dtype before", props[col].dtype)
            
            # 判断是否是int类型
            isInt = False
            mmax = props[col].max()
            mmin = props[col].min()
            
            # Integer does not support NA, therefore Na needs to be filled
            if not np.isfinite(props[col]).all():
                NAlist.append(col)
                props[col].fillna(-999, inplace=True) # 用-999填充
                
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = np.fabs(props[col] - asint)
            result = result.sum()
            if result < 0.01: # 绝对误差和小于0.01认为可以转换的，要根据task修改
                isInt = True
            
            # make interger / unsigned Integer datatypes
            if isInt:
                if mmin >= 0: # 最小值大于0，转换成无符号整型
                    if mmax <= 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mmax <= 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mmax <= 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else: # 转换成有符号整型
                    if mmin > np.iinfo(np.int8).min and mmax < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mmin > np.iinfo(np.int16).min and mmax < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mmin > np.iinfo(np.int32).min and mmax < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mmin > np.iinfo(np.int64).min and mmax < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)  
            else: # 注意：这里对于float都转换成float16，需要根据你的情况自己更改
                props[col] = props[col].astype(np.float16)
            
            # print("dtype after", props[col].dtype)
            # print("********************************")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [5]:
# 处理id字段
train['order_detail_id'] = train['order_detail_id'].astype(np.uint32)
train['order_id'] = train['order_id'].astype(np.uint32)
train['customer_id'] = train['customer_id'].astype(np.uint32)
train['goods_id'] = train['goods_id'].astype(np.uint32)
train['goods_class_id'] = train['goods_class_id'].astype(np.uint32)
train['member_id'] = train['member_id'].astype(np.uint32)
# 处理状态字段，这里同时处理空值，将空值置为0
train['order_status'] = train['order_status'].astype(np.uint8)
train['goods_has_discount'] = train['goods_has_discount'].astype(np.uint8)
train["is_member_actived"].fillna(0, inplace=True)
train["is_member_actived"]=train["is_member_actived"].astype(np.int8)
train["member_status"].fillna(0, inplace=True)
train["member_status"]=train["member_status"].astype(np.int8)
train["customer_gender"].fillna(0, inplace=True)
train["customer_gender"]=train["customer_gender"].astype(np.int8)
train['is_customer_rate'] = train['is_customer_rate'].astype(np.uint8)
train['order_detail_status'] = train['order_detail_status'].astype(np.uint8)
# 处理日期
train['goods_list_time']=pd.to_datetime(train['goods_list_time'],format="%Y-%m-%d")
train['order_pay_time']=pd.to_datetime(train['order_pay_time'],format="%Y-%m-%d")
train['goods_delist_time']=pd.to_datetime(train['goods_delist_time'],format="%Y-%m-%d")
# 检查内存使用
mem_train = train.memory_usage(index=True).sum()
mem_test=test.memory_usage(index=True).sum()
print(u"训练集使用内容 "+ str(mem_train/ 1024**2)+" MB")
print(u"测试集使用内存 "+ str(mem_test/ 1024**2)+" MB")

训练集使用内容 349.8006982803345 MB
测试集使用内存 24.200225830078125 MB


In [6]:
train['customer_city_id'] = LabelEncoder().fit_transform(train['customer_city'].astype(str))
train['customer_province_id'] = LabelEncoder().fit_transform(train['customer_province'].astype(str))

# 纯时间序列滑窗
训练集和测试集构造思路：
- 训练集
	- 训练日期截止分别为2013.6.6、2013.6.20、2013.7.1、2013.7.15
	- 标签为训练截止日期开始的30天有没有下过单
	- 标签通过切片的形式构造
    - 把4个部分训练集拼起来
- 验证集
	- 验证数据到2013.8.1
	- 标签为2013.8.1开始的30天有没有下过单
	- 标签时间2013.8.1-2013.8.30
- 测试集
	- 测试数据到2013.8.31
	- 没有标签

In [7]:
train.head()

,order_detail_id,order_id,order_total_num,order_amount,order_total_payment,order_total_discount,order_pay_time,order_status,order_count,is_customer_rate,...,is_member_actived,goods_id,goods_class_id,goods_price,goods_status,goods_has_discount,goods_list_time,goods_delist_time,customer_city_id,customer_province_id
0,1000000,1000000,1.0,239.9,96.9,0.0,2012-11-01 00:10:56,6,1.0,0,...,0,998,998,54.909289,1.0,0,2014-10-25 11:08:07,2014-11-01 11:08:07,53,4
1,1001530,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0,...,0,1953,1953,45.961352,0.0,1,2013-08-28 17:27:50,2013-09-01 00:38:17,322,3
2,1001531,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0,...,0,1083,1083,53.035439,1.0,0,2014-10-29 18:21:05,2014-11-05 18:21:05,322,3
3,1001532,1001328,3.0,180.0,89.7,0.0,2013-08-31 22:06:35,6,1.0,0,...,0,1013,1013,46.046917,1.0,1,2014-10-25 11:00:00,2014-11-01 11:00:00,185,20
4,1001533,1001329,1.0,159.9,65.9,0.0,2013-08-31 21:33:36,6,1.0,0,...,0,1628,1628,50.722161,1.0,0,2014-10-23 15:35:33,2014-10-30 15:35:33,53,4


In [8]:
len(train['customer_id'].unique())

1585986

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1585986 entries, 1000000 to 2826574
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   result  1585986 non-null  float64
dtypes: float64(1)
memory usage: 24.2 MB


In [10]:
test.head()

,result
customer_id,
1000000,0.0
1000014,0.0
1000034,0.0
1000046,0.0
1000048,0.0


从上面的比对可以看出，这里要预测的是全体用户未来一个月的购买情况，训练集和测试集的`id`完全重合，没有需要特别处理的地方

## 构造时间滑窗特征

时间滑窗构造
- [已完成] 考虑到按周、按月的订单规律性变化，时间滑窗往往是7和30的倍数
- [已完成] 一般选取的特征是滑窗中的基本统计值，最大、最小、均值、中位数、求和等等
- [未完成] 节假日信息、影响周期的时间滑窗整合

### 每日付款金额

In [11]:
# 将用户下单金额按天进行汇总
# df = train[train.order_status<101][train.order_pay_time>'2013-02-01']
df = train[train.order_pay_time>'2013-02-01']
df['date'] = pd.DatetimeIndex(df['order_pay_time']).date
df_payment = df[['customer_id','date','order_total_payment']]

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
len(df_payment['customer_id'].unique())

685471

注意，成功交易的客户数量不等于全部客户数量，说明有相当一部分客户虽然下过单，但是没有成功的订单，那么这些客户自然应当算在训练集之外。
数据合并时，由于`test.csv`中，已经设置了默认0值，只需要和训练后的预测标签做一个`left join`就可以了

In [13]:
df_payment = df_payment.groupby(['date','customer_id']).agg({'order_total_payment': ['sum']})
df_payment.columns = ['day_total_payment']
df_payment.reset_index(inplace=True)

In [14]:
df_payment = df_payment.set_index(
    ["customer_id", "date"])[["day_total_payment"]].unstack(level=-1).fillna(0)
df_payment.columns = df_payment.columns.get_level_values(1)

In [15]:
df_payment.head()

date,2013-02-01,2013-02-02,2013-02-03,2013-02-04,2013-02-05,2013-02-06,2013-02-07,2013-02-08,2013-02-09,2013-02-10,...,2013-08-22,2013-08-23,2013-08-24,2013-08-25,2013-08-26,2013-08-27,2013-08-28,2013-08-29,2013-08-30,2013-08-31
customer_id,,,,,,,,,,,,,,,,,,,,,
1000014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 每日购买商品数量

In [16]:
df_goods = df[['customer_id','date','order_total_num']]
df_goods = df_goods.groupby(['date','customer_id']).agg({'order_total_num': ['sum']})
df_goods.columns = ['day_total_num']
df_goods.reset_index(inplace=True)
df_goods = df_goods.set_index(
    ["customer_id", "date"])[["day_total_num"]].unstack(level=-1).fillna(0)
df_goods.columns = df_goods.columns.get_level_values(1)

该场景每天都有成交记录，这样就不需要考虑生成完整时间段填充的问题

In [17]:
# 这是一个时间滑窗函数，获得dt之前minus天以来periods的dataframe，以便进一步计算
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

1. 构造dataset这里有个取巧的地方，因为要预测的9月份除了开学季以外不是非常特殊的月份，因此主要考虑近期的因素，数据集的开始时间也是2月1日，尽量避免了双十一、元旦假期的影响，当然春节假期继续保留。同时，构造数据集的时候保留了customer_id，主要为了与其它特征做整合。
2. 通过一个函数整合付款金额和商品数量的时间滑窗，主要是因为分开做到时候合并占用内存更大，并且函数最后在返回值处做了内存优化，用时间代价尽可能避免内存溢出。

In [18]:
def prepare_dataset(df_payment, df_goods, t2018, is_train=True):
    X = {}
    # 整合用户id
    tmp = df_payment.reset_index()
    X['customer_id'] = tmp['customer_id']
    # 消费特征
    print('Preparing payment feature...')
    for i in [7,14,30,49,60,91,120]:
        tmp_1 = get_timespan(df_payment, t2018, i, i)
        # X['diff_%s_mean' % i] = tmp_1.diff(axis=1).mean(axis=1).values
        # X['mean_%s_decay' % i] = (tmp_1 * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s' % i] = tmp_1.mean(axis=1).values
        X['median_%s' % i] = tmp_1.median(axis=1).values
        # X['min_%s' % i] = tmp_1.min(axis=1).values
        X['max_%s' % i] = tmp_1.max(axis=1).values
        # X['std_%s' % i] = tmp_1.std(axis=1).values
        X['sum_%s' % i] = tmp_1.sum(axis=1).values

        tmp_2 = get_timespan(df_payment, t2018 + timedelta(weeks=-1), i, i)
        # X['diff_%s_mean_2' % i] = tmp_2.diff(axis=1).mean(axis=1).values
        # X['mean_%s_decay_2' % i] = (tmp_2 * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s_2' % i] = tmp_2.mean(axis=1).values
        X['median_%s_2' % i] = tmp_2.median(axis=1).values
        # X['min_%s_2' % i] = tmp_2.min(axis=1).values
        X['max_%s_2' % i] = tmp_2.max(axis=1).values
        # X['std_%s_2' % i] = tmp_2.std(axis=1).values

        tmp_3 = get_timespan(df_payment, t2018, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp_3 != 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp_3 != 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp_3 != 0) * np.arange(i, 0, -1)).max(axis=1).values

    # 对此处进行微调，主要考虑近期因素
    for i in range(1, 4):
        X['day_%s_2018' % i] = get_timespan(df_payment, t2018, i*30, 30).sum(axis=1).values

    for i in range(7):
        X['mean_4_dow{}_2013'.format(i)] = get_timespan(df_payment, t2018, 56-i*2, 4, freq='14D').mean(axis=1).values
        # X['mean_20_dow{}_2013'.format(i)] = get_timespan(df, t2018, 140-i, 20, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2013'.format(i)] = get_timespan(df_payment, t2018, 140-i*2, 10, freq='14D').mean(axis=1).values
    
    # 商品数量特征，这里故意把时间和消费特征错开，提高时间滑窗的覆盖面
    print('Preparing num feature...')
    for i in [3,21,35,49,70,84,105]:
            tmp_1 = get_timespan(df_goods, t2018, i, i)
            # X['goods_diff_%s_mean' % i] = tmp_1.diff(axis=1).mean(axis=1).values
            # X['goods_mean_%s_decay' % i] = (tmp_1 * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
            X['goods_mean_%s' % i] = tmp_1.mean(axis=1).values
            X['goods_median_%s' % i] = tmp_1.median(axis=1).values
            # X['goods_min_%s' % i] = tmp_1.min(axis=1).values
            X['goods_max_%s' % i] = tmp_1.max(axis=1).values
            # X['goods_std_%s' % i] = tmp_1.std(axis=1).values
            X['goods_sum_%s' % i] = tmp_1.sum(axis=1).values
    
            tmp_2 = get_timespan(df_goods, t2018 + timedelta(weeks=-1), i, i)
            # X['goods_diff_%s_mean_2' % i] = tmp_2.diff(axis=1).mean(axis=1).values
            # X['goods_mean_%s_decay_2' % i] = (tmp_2 * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
            X['goods_mean_%s_2' % i] = tmp_2.mean(axis=1).values
            X['goods_median_%s_2' % i] = tmp_2.median(axis=1).values
            # X['goods_min_%s_2' % i] = tmp_2.min(axis=1).values
            X['goods_max_%s_2' % i] = tmp_2.max(axis=1).values
            X['goods_sum_%s_2' % i] = tmp_2.sum(axis=1).values
    
            tmp_3 = get_timespan(df_goods, t2018, i, i)
            X['goods_has_sales_days_in_last_%s' % i] = (tmp_3 > 0).sum(axis=1).values
            X['goods_last_has_sales_day_in_last_%s' % i] = i - ((tmp_3 > 0) * np.arange(i)).max(axis=1).values
            X['goods_first_has_sales_day_in_last_%s' % i] = ((tmp_3 > 0) * np.arange(i, 0, -1)).max(axis=1).values


    # 对此处进行微调，主要考虑近期因素
    for i in range(1, 4):
        X['goods_day_%s_2018' % i] = get_timespan(df_goods, t2018, i*28, 28).sum(axis=1).values

    for i in range(7):
        X['goods_mean_4_dow{}_2013'.format(i)] = get_timespan(df_goods, t2018, 56-i*2, 4, freq='14D').mean(axis=1).values
        # X['mean_20_dow{}_2013'.format(i)] = get_timespan(df, t2018, 140-i, 20, freq='7D').mean(axis=1).values
        X['goods_mean_20_dow{}_2013'.format(i)] = get_timespan(df_goods, t2018, 140-i*2, 10, freq='14D').mean(axis=1).values

    X = pd.DataFrame(X)
    
    reduce_mem_usage(X)
    
    if is_train:
        # 这样转换之后，打标签直接用numpy切片就可以了
        # 当然这里前提是确认付款总额没有负数的问题
        y = df_goods[pd.date_range(t2018, periods=30)].max(axis=1).values
        y[y > 0] = 1
        return X, y
    return X

## 生成滑窗数据
此处对内存占用要求非常高，尽量分步操作避免crash，即使如此，内存溢出仍然频繁出现，通过下面的办法进一步压缩数据集大小

### 生成训练集
对此处进行优化，把训练集不曾涉及到的新增用户剔除，也就是说，模型研究的是已有用户的购买行为变化趋势。如果一个用户在8月中旬才第一次下单，那么就不会被放到训练集/验证集中，只会出现在测试集中。

内存限制的情况下，训练集和验证集时间的选取会比较纠结，既要考虑月份，也要考虑星期。

In [19]:
def make_slides(train,df_part,begin,end,column):
    # 将用户下单金额按天进行汇总
    # df = train[train.order_status <= 6][train.order_pay_time > '2013-02-01']
    df = train[train.order_pay_time > begin][train.order_pay_time < end]
    df = pd.merge(df,df_part,how='inner')
    df['date'] = pd.DatetimeIndex(df['order_pay_time']).date
    df = df[['customer_id', 'date', column]]
    df = df.groupby(['date', 'customer_id']).agg({column: ['sum']})
    df.columns = ['day_' + column]
    df.reset_index(inplace=True)
    df = df.set_index(
        ["customer_id", "date"])[['day_' + column]].unstack(level=-1).fillna(0)
    df.columns = df.columns.get_level_values(1)
    return df

#### part1
2013.06.06之前的用户，在2013.06.06后30天的时间滑窗

In [20]:
df_0606 = train[train.order_pay_time > '2013-01-01'][train.order_pay_time <= '2013-06-06'][['customer_id']]
# 删除重复行
df_0606 = df_0606.drop_duplicates(['customer_id'])
df_part1_partment = make_slides(train,df_0606,'2013-01-01','2013-07-06','order_total_payment')
df_part1_goods = make_slides(train,df_0606,'2013-01-01','2013-07-06','order_total_num')
X_part1, y_part1 = prepare_dataset(df_part1_partment, df_part1_goods, date(2013, 6, 6))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 1057.7543182373047 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  219.39694023132324  MB
This is  20.741767388568775 % of the initial size


In [21]:
X_part1.to_pickle('work/X_part1.pkl')
np.save("work/y_part1.npy", y_part1)

In [22]:
del df_0606
del df_part1_partment
del df_part1_goods
del X_part1
del y_part1
gc.collect()

21

#### part2
2013.06.20之前的用户，在2013.06.20后30天的时间滑窗

In [23]:
df_0620 = train[train.order_pay_time > '2013-01-15'][train.order_pay_time <= '2013-06-20'][['customer_id']]
# 删除重复行
df_0620 = df_0620.drop_duplicates(['customer_id'])
df_part2_partment = make_slides(train,df_0620,'2013-01-15','2013-07-20','order_total_payment')
df_part2_goods = make_slides(train,df_0620,'2013-01-15','2013-07-20','order_total_num')
X_part2, y_part2 = prepare_dataset(df_part2_partment, df_part2_goods, date(2013, 6, 20))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 1051.5419464111328 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  215.2195339202881  MB
This is  20.46704219977368 % of the initial size


In [24]:
X_part2.to_pickle('work/X_part2.pkl')
np.save("work/y_part2.npy", y_part2)

In [25]:
del df_0620
del df_part2_partment
del df_part2_goods
del X_part2
del y_part2
gc.collect()

21

#### part3
2013.07.01之前的用户，在2013.07.01后30天的时间滑窗

In [26]:
df_0701 = train[train.order_pay_time > '2013-01-26'][train.order_pay_time <= '2013-07-01'][['customer_id']]
# 删除重复行
df_0701 = df_0701.drop_duplicates(['customer_id'])
df_part3_partment = make_slides(train,df_0701,'2013-01-26','2013-07-31','order_total_payment')
df_part3_goods = make_slides(train,df_0701,'2013-01-26','2013-07-31','order_total_num')
X_part3, y_part3 = prepare_dataset(df_part3_partment, df_part3_goods, date(2013, 7, 1))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 801.8140411376953 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  165.20903396606445  MB
This is  20.604407691794606 % of the initial size


In [27]:
X_part3.to_pickle('work/X_part3.pkl')
np.save("work/y_part3.npy", y_part3)

In [28]:
del df_0701
del df_part3_partment
del df_part3_goods
del X_part3
del y_part3
gc.collect()

21

#### part4
2013.07.15之前的用户，在2013.07.15后30天的时间滑窗

In [29]:
df_0715 = train[train.order_pay_time > '2013-02-10'][train.order_pay_time <= '2013-07-15'][['customer_id']]
# 删除重复行
df_0715 = df_0715.drop_duplicates(['customer_id'])
df_part4_partment = make_slides(train,df_0715,'2013-02-10','2013-08-16','order_total_payment')
df_part4_goods = make_slides(train,df_0715,'2013-02-10','2013-08-16','order_total_num')
X_part4, y_part4 = prepare_dataset(df_part4_partment, df_part4_goods, date(2013, 7, 15))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 526.6256866455078 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  107.06136322021484  MB
This is  20.32968879702254 % of the initial size


In [30]:
X_part4.to_pickle('work/X_part4.pkl')
np.save("work/y_part4.npy", y_part4)

In [31]:
del df_0715
del df_part4_partment
del df_part4_goods
del X_part4
del y_part4
gc.collect()

21

In [32]:
print("Preparing Train dataset...")
X_l, y_l = [], []
X_part1 = pd.read_pickle('work/X_part1.pkl')
y_part1 = np.load('work/y_part1.npy')
X_l.append(X_part1)
y_l.append(y_part1)
X_part2 = pd.read_pickle('work/X_part2.pkl')
y_part2 = np.load('work/y_part2.npy')
X_l.append(X_part2)
y_l.append(y_part2)
X_part3 = pd.read_pickle('work/X_part3.pkl')
y_part3 = np.load('work/y_part3.npy')
X_l.append(X_part3)
y_l.append(y_part3)
X_part4 = pd.read_pickle('work/X_part4.pkl')
y_part4 = np.load('work/y_part4.npy')
X_l.append(X_part4)
y_l.append(y_part4)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
print('done!')
del X_l
del y_l
del X_part1
del y_part1
del X_part2
del y_part2
del X_part3
del y_part3
del X_part4
del y_part4
gc.collect()
X_train.to_pickle('work/X_train.pkl')
np.save("work/y_train.npy", y_train)

Preparing Train dataset...
done!


### 生成验证集
2013.08.01之前的用户，在2013.08.01后30天的时间滑窗

In [33]:
print("Preparing Validation dataset...")
df_0801 = train[train.order_pay_time > '2013-03-01'][train.order_pay_time <= '2013-08-01'][['customer_id']]
# 删除重复行
df_0801 = df_0801.drop_duplicates(['customer_id'])
df_part5_partment = make_slides(train,df_0801,'2013-03-01','2013-08-31','order_total_payment')
df_part5_goods = make_slides(train,df_0801,'2013-03-01','2013-08-31','order_total_num')
X_val, y_val = prepare_dataset(df_part5_partment, df_part5_goods, date(2013, 8, 1))

Preparing Validation dataset...


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 561.9365081787109 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  113.85399150848389  MB
This is  20.26100633281425 % of the initial size


In [34]:
X_val.to_pickle('work/X_val.pkl')
np.save("work/y_val.npy", y_val)

In [35]:
del df_0801
del df_part5_partment
del df_part5_goods
gc.collect()

21

### 生成测试集

In [36]:
df_0901 = train[train.order_pay_time > '2013-03-01'][train.order_pay_time < '2013-09-01'][['customer_id']]
# 删除重复行
df_0901 = df_0901.drop_duplicates(['customer_id'])
df_test_partment = make_slides(train,df_0901,'2013-04-01','2013-09-01','order_total_payment')
df_test_goods = make_slides(train,df_0901,'2013-04-01','2013-09-01','order_total_num')
X_test = prepare_dataset(df_test_partment, df_test_goods, date(2013, 9, 1),is_train=False)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 672.3984222412109 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  137.15829181671143  MB
This is  20.398366099602228 % of the initial size


In [37]:
X_test.to_pickle('work/X_test.pkl')

至此，整合了用户id的消费时间滑窗就完整了

# 用户基本特征

把地域、会员这些基本信息整合到训练数据中

In [38]:
df_customer = train[['member_id','customer_id','customer_gender','customer_province_id','customer_city_id','is_member_actived']]
# 删除重复行
df_customer = df_customer.drop_duplicates(['customer_id'])
df_customer.head()

,member_id,customer_id,customer_gender,customer_province_id,customer_city_id,is_member_actived
0,0,1000000,0,4,53,0
1,0,1001324,0,3,322,0
3,0,1001325,0,20,185,0
4,0,1001326,0,4,53,0
5,0,1001327,0,4,53,0


In [39]:
X_train = pd.merge(X_train,df_customer,how='left')
X_val = pd.merge(X_val,df_customer,how='left')
X_test = pd.merge(X_test,df_customer,how='left')

In [40]:
X_train.to_pickle('X_train.pkl')
X_val.to_pickle('X_val.pkl')
X_test.to_pickle('X_test.pkl')

# 模型训练

In [41]:
X_train = pd.read_pickle('X_train.pkl')
X_val = pd.read_pickle('X_val.pkl')
X_test = pd.read_pickle('X_test.pkl')

y_train = np.load("work/y_train.npy")
y_val = np.load("work/y_val.npy")

In [42]:
X_l, y_l = [], []
xx_l,yy_l=[],[]
X_l.append(X_train)
y_l.append(y_train)

xx_l.append(X_val)
yy_l.append(y_val)

X_train_cv = pd.concat(X_l, axis=0)
y_train_cv = np.concatenate(y_l, axis=0)

X_val_cv = pd.concat(xx_l, axis=0)
y_val_cv = np.concatenate(yy_l, axis=0)


del X_l
del y_l
del xx_l
del yy_l
gc.collect()

7948

**由于正负样本不均衡，因此scale_pos_weight设置为 sum(negative cases) / sum(positive cases)**

In [43]:
import xgboost as xgb

xgb_train = xgb.DMatrix(X_train_cv.values, label=y_train_cv)
xgb_val = xgb.DMatrix(X_val_cv.values, label=y_val_cv)
xgb_test = xgb.DMatrix(X_test.values)
y=y_train_cv

params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'gamma': 0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 8,  # 构建树的深度，越大越容易过拟合
    'alpha': 0,   # L1正则化系数
    'lambda': 10,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,  # 随机采样训练样本
    'colsample_bytree': 0.5,  # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,  
    'eta': 0.03, 
    'seed': 1000,
    'nthread': 8,  
    'missing': 1,
    'scale_pos_weight': (np.sum(y==0)/np.sum(y==1))  ,# 用来处理正负样本不均衡的问题,通常取：sum(negative cases) / sum(positive cases)
    'eval_metric': "auc"
}

plst = list(params.items())
num_rounds = 200  # 迭代次数
watchlist = [(xgb_train, 'train'),(xgb_val,'valid')]
model = xgb.train(plst, xgb_train, num_rounds, watchlist)
model.save_model('model.xgb')
pred_value = model.predict(xgb_test)

[16:09:23] WARNING: ../src/learner.cc:541: 
Parameters: { missing, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.78205	valid-auc:0.73007
[1]	train-auc:0.78570	valid-auc:0.73243
[2]	train-auc:0.80639	valid-auc:0.75874
[3]	train-auc:0.80430	valid-auc:0.75572
[4]	train-auc:0.81058	valid-auc:0.76400
[5]	train-auc:0.80928	valid-auc:0.76143
[6]	train-auc:0.80795	valid-auc:0.76005
[7]	train-auc:0.81139	valid-auc:0.76408
[8]	train-auc:0.81016	valid-auc:0.76280
[9]	train-auc:0.81288	valid-auc:0.76653
[10]	train-auc:0.81188	valid-auc:0.76589
[11]	train-auc:0.81170	valid-auc:0.76568
[12]	train-auc:0.81375	valid-auc:0.76880
[13]	train-auc:0.81317	valid-auc:0.76809
[14]	train-auc:0.81505	valid-auc:0.77034
[15]	train-auc:0.81641	valid-auc:0.77228
[16]	train-auc:0.81

# 加载模型

In [60]:
# model=xgb.Booster(params)
# model.load_model('model.xgb') 
# pred_value = model.predict(xgb_test)

# 结果预测

In [45]:
test_pred = pred_value # 本来做了bagging，效果不好

In [46]:
print("Preparing Predict dataset...")

print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    {    
    "customer_id": X_test.customer_id, 
    "pred": y_test
    }
)
df_preds.head()

Preparing Predict dataset...
Making submission...


,customer_id,pred
0,1000014,0.546633
1,1000046,0.256358
2,1000069,0.545569
3,1000105,0.917949
4,1000109,0.592108


In [52]:


sub = pd.read_csv('data/data19383/submission.csv')

# 合并预测结果
submission = pd.merge(sub, df_preds, on='customer_id', how='left')
submission.fillna(0,inplace=True)
submission = submission[['customer_id','pred']]
submission.rename(columns={'customer_id':'customer_id','pred':'result'}, inplace=True)

# 将概率值转换为用户是否购买的标签
def f(x):
    if x <= 0.30:   # 调整阈值
        return 0
    else:
        return 1
    return x
submission['result'] = submission['result'].map(f)

# 保存结果
submission.to_csv('submission.csv',index=False)

In [53]:
submission.sum()

customer_id    3033146322233
result                334857
dtype: int64

	未来，我们可以继续探索更为先进的机器学习模型，并使用更多的评估指标来更准确地评估模型的性能。此外，我们还可以尝试使用更为复杂的特征工程方法来处理数据集，以提升模型的性能。
	总的来说，本实验提供了一种使用XGBoost模型进行商家订单数据分析的方法。


# 参考资料
[用户购买预测时间滑窗特征构建](https://aistudio.baidu.com/aistudio/projectdetail/276829)